In [14]:
import requests
import json
import pandas as pd
import datetime as dt
import numpy as np

In [15]:
sw8 = pd.read_csv('./data/dongjak_sw8.csv')
cfdata = pd.read_csv('./data/cfdata.csv')

In [16]:
#sw8.sort_values(by=['nb_num'],axis=0,ascending=False).head(30)

In [17]:
date = pd.to_datetime(cfdata['DCBYMD'],format='%Y-%m-%d')
opdate = pd.to_datetime(cfdata['APVPERMYMD'],format='%Y-%m-%d')
cfdata['DCBYMD'] = date
cfdata['APVPERMYMD'] = opdate
cfdata.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24541 entries, 0 to 24540
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   Unnamed: 0   24541 non-null  int64         
 1   APVPERMYMD   24541 non-null  datetime64[ns]
 2   TRDSTATEGBN  24541 non-null  int64         
 3   TRDSTATENM   24541 non-null  object        
 4   DCBYMD       8624 non-null   datetime64[ns]
 5   SITEAREA     23486 non-null  object        
 6   RDNWHLADDR   24541 non-null  object        
 7   BPLCNM       24541 non-null  object        
 8   X            24541 non-null  float64       
 9   Y            24541 non-null  float64       
 10  region       24541 non-null  object        
dtypes: datetime64[ns](2), float64(2), int64(2), object(5)
memory usage: 2.1+ MB


In [20]:
#cfdata = cfdata.drop(['UPTAENM'],axis=1)

In [21]:
#cfdata = cfdata.drop(['Unnamed: 0'],axis=1)

In [22]:
#2017년 이후 폐업한 가게
closed = cfdata[cfdata['TRDSTATEGBN']==3]
#closed = closed.set_index('DCBYMD')
closed2017 = closed.loc[(closed.DCBYMD.dt.year == 2017)]
closed2018 = closed.loc[(closed.DCBYMD.dt.year == 2018)]
closed2019 = closed.loc[(closed.DCBYMD.dt.year == 2019)]
closed2020 = closed.loc[(closed.DCBYMD.dt.year == 2020)]

In [23]:
#오픈 가게 데이터
opening = cfdata[cfdata['TRDSTATEGBN']==1]

In [24]:
fdata = pd.concat([closed2017,closed2018,closed2019,closed2020,opening])
fdata = fdata.dropna(axis=0,subset=['X'])
fdata = fdata.reset_index(drop=True)
addr = fdata['RDNWHLADDR']
addr=addr.reset_index(drop=True)
DCBYMD = fdata['DCBYMD']
DCBYMD=DCBYMD.reset_index(drop=True)
region = fdata['region']
region=region.reset_index(drop=True)

x = fdata['X']
y = fdata['Y']
x=x.reset_index(drop=True)
y=y.reset_index(drop=True)

x=x.astype('str')
y=y.astype('str')

fdata

,APVPERMYMD,TRDSTATEGBN,TRDSTATENM,DCBYMD,SITEAREA,RDNWHLADDR,BPLCNM,X,Y,region
0,2013-11-13,3,폐업,2017-10-11,22.53,서울특별시 강동구 양재대로87길 11 (성내동),탭플레이 커피,127.134306,37.525438,강동구
1,2015-07-08,3,폐업,2017-07-10,22.00,"서울특별시 강동구 진황도로27길 23, 1층 (천호동)",아카페,127.131877,37.539416,강동구
2,2014-03-20,3,폐업,2017-07-17,74.21,"서울특별시 강동구 구천면로 428 (명일동, 명일센트로빌 205호)",CAFE TENUTO 테누토,127.146158,37.550130,강동구
3,2014-12-05,3,폐업,2017-06-02,337.19,"서울특별시 강동구 강동대로 205, 1층 (성내동)",할리스커피 올림픽공원북문점,127.129210,37.524394,강동구
4,2013-01-03,3,폐업,2017-05-18,30.00,서울특별시 강동구 풍성로63길 84 (둔촌동),도우드림,127.140339,37.531725,강동구
...,...,...,...,...,...,...,...,...,...,...
24536,2011-03-02,1,영업/정상,NaT,98.10,"서울특별시 서초구 강남대로39길 15-7 (서초동,브니엘빌딩 1층)",카페 엘리,127.032416,37.485785,서초구
24537,2010-09-16,1,영업/정상,NaT,50.00,서울특별시 서초구 바우뫼로41길 8 (양재동),카페레즈,127.041496,37.481075,서초구
24538,2010-05-27,1,영업/정상,NaT,142.50,"서울특별시 서초구 사평대로 106, 1층 (반포동)",마켓 비노플라워,126.996156,37.498874,서초구
24539,2009-10-29,1,영업/정상,NaT,62.00,서울특별시 서초구 바우뫼로41길 88 (양재동),까페원더,127.039916,37.484342,서초구


In [10]:
#url1 = 'https://dapi.kakao.com/v2/local/geo/transcoord.json?x=' + x[2283] +'&y='+ y[2283] +'&input_coord=TM&output_coord=WGS84'
#headers = {"Authorization":"KakaoAK c99fe16b66a1887f51edaffb856e5997"}
#result1 = json.loads(str(requests.get(url1, headers=headers).text))
#len(result1)

In [11]:
cf_x = []
cf_y = []

for i in range(len(x)):
    #2017~ 가게 주소 변환(TM-WGS84)

    url1 = 'https://dapi.kakao.com/v2/local/geo/transcoord.json?x=' + x[i] +'&y='+ y[i] +'&input_coord=TM&output_coord=WGS84'
    headers = {"Authorization": "KakaoAK 6341d1e76477fbac498a469adc87108d"}
    result1 = json.loads(str(requests.get(url1, headers=headers).text))
    
    cf_x.append(result1['documents'][0]['x'])   #카페 데이터의 좌표(WGS84)
    cf_y.append(result1['documents'][0]['y'])

    
fdata['X'] = cf_x
fdata['Y'] = cf_y
fdata

,APVPERMYMD,TRDSTATEGBN,TRDSTATENM,DCBYMD,SITEAREA,RDNWHLADDR,BPLCNM,X,Y,region
0,2013-11-13,3,폐업,2017-10-11,22.53,서울특별시 강동구 양재대로87길 11 (성내동),탭플레이 커피,127.134306,37.525438,강동구
1,2015-07-08,3,폐업,2017-07-10,22.00,"서울특별시 강동구 진황도로27길 23, 1층 (천호동)",아카페,127.131877,37.539416,강동구
2,2014-03-20,3,폐업,2017-07-17,74.21,"서울특별시 강동구 구천면로 428 (명일동, 명일센트로빌 205호)",CAFE TENUTO 테누토,127.146158,37.550130,강동구
3,2014-12-05,3,폐업,2017-06-02,337.19,"서울특별시 강동구 강동대로 205, 1층 (성내동)",할리스커피 올림픽공원북문점,127.129210,37.524394,강동구
4,2013-01-03,3,폐업,2017-05-18,30.00,서울특별시 강동구 풍성로63길 84 (둔촌동),도우드림,127.140339,37.531725,강동구
...,...,...,...,...,...,...,...,...,...,...
24536,2011-03-02,1,영업/정상,NaT,98.10,"서울특별시 서초구 강남대로39길 15-7 (서초동,브니엘빌딩 1층)",카페 엘리,127.032416,37.485785,서초구
24537,2010-09-16,1,영업/정상,NaT,50.00,서울특별시 서초구 바우뫼로41길 8 (양재동),카페레즈,127.041496,37.481075,서초구
24538,2010-05-27,1,영업/정상,NaT,142.50,"서울특별시 서초구 사평대로 106, 1층 (반포동)",마켓 비노플라워,126.996156,37.498874,서초구
24539,2009-10-29,1,영업/정상,NaT,62.00,서울특별시 서초구 바우뫼로41길 88 (양재동),까페원더,127.039916,37.484342,서초구


In [13]:
fdata.to_csv('./data/cfdata.csv',encoding='utf-8')